# Chapter 4

Having the maturity and the pay-off in different objects is not the most optimal thing. It makes more sense to have it all inside a class. 

### First Solution

In [46]:
class VanillaOption():
    def __init__(self,ThePayOff_,Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self,Spot):
        return self.__ThePayOff(Spot)

Now we put it all together

In [2]:
from random import random
from math import exp, sqrt, log

In [14]:
def SimpleMonteCarlo3(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    Expiry = TheOption.GetExpiry()
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean      

In [4]:
def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    result -= 6
    return result

def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1:
        x = 2*random() - 1
        y = 2*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

In [5]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

In [6]:
class PayOffCall(PayOff):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

In [47]:
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
callPrice = SimpleMonteCarlo3(TheOption,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.82, for 10000 paths


However this solution is not the optimal since we moght want to handle the memory management and other issues of the payoff separately from the otpion class. For this reason we will use the bridge  design pattern in which we separate the memory management issues of the payoff from other classes.

### The bridge

Now we will use the PayOffBridge to build an object between the pay-off and the option class to add extra functionalities like memory management and other features.

In [97]:
class PayOffBridge():
    def __init__(self,PayOff_):
        self.__PayOff = PayOff_
        
    def __del__(self):
        del self.__PayOff
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self.__PayOff = deepcopy(InnerPayOff)
    
    # We cannot overload the operator =  in python
    
    def __call__(self,Spot):
        return self.__PayOff(Spot)

Now we will redefine the vanilla option class.

In [54]:
class VanillaOption():
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)

In [59]:
callPayOff = PayOffBridge(PayOffCall(40))
TheOption = VanillaOption(callPayOff,0.5)
callPrice = SimpleMonteCarlo3(TheOption,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.78, for 10000 paths


### A parameters class

We might want to know some characteristics of our parameters, such as the integral or the mean. The most sensible way to treat that is as an object.

This is the basic parent class for all the parameters.

In [224]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

This is an exemple of the parameters class implemented for a constant, which is the most basic and easy to implement type of parameters.

In [98]:
class ParametersConstant(ParametersInner):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare

The book adds a bridge again to add new functionalities such as the mean or the RMSE to the each parameter class, inhereting from the ParametersInner. However we can do that also with a multiple inheretance.

In [223]:
class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    

Then we will have this class.

In [188]:
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare

The result is a cleaner way of doing it instead of the previous approach we did for the pay off. We can also reconstruct the pay-off. It would be done this way.

In [212]:
class PayOffBridge():
        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
    
    # We cannot overload the operator =  in python

In [213]:
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

Now we will put it everything together with a modified MonteCarlo routine.

In [214]:
def SimpleMonteCarlo4(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    Expiry = TheOption.GetExpiry()
    
    variance = Vol.IntegralSquare(0,Expiry)
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r.Integral(0,Expiry) + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r.Integral(0,Expiry))
    return mean    

In [222]:
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
r = ParametersConstant(0.1)
Vol = ParametersConstant(0.2)
callPrice = SimpleMonteCarlo4(TheOption,42,Vol,r,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.83, for 10000 paths


### Exercises

**Exercise 4.1** Test how fast new is on your computer and compiler. Do this by using it to allocate an array of doubles, ten thousand times. See how the speed varies with array size. If you have more than one compiler see how they compare. Note that you can time things using the clock() function.

Since we are not using **new**, beaucause of the implementation in python it really does not make sense to do this exercise.

**Exercise 4.2** Find out an auto_ptr. Observe that it cannot be copied in the usual sense of copying. Show that a a class with an auto_ptr data member will need a copy constructor but not a destructor.

We are not using C++ and pointers so it does not make sense to do this exercise.

**Exercise 4.3** Implement a piecewise-constant parameters class.

A piecewise-constant it is just a constant that has different values in different intervals, for example we could have a parameter that is 0 from day 0 to day 100 and then 1 from day 101 to inf.

It takes as inputs two numpy arrays, one with the values of the parameters and another with the intervals.

In [108]:
import numpy as np

In [207]:
class ParametersPiecewiseConstant(ParametersInner,Parameters):
    def __init__(self,constants,first,second):
        self.__Constants = constants
        self.__ConstantsSquare = np.power(self.__Constants, 2)
        self.__First = first
        self.__Second = second
    
    def Integral(self, time1, time2):
        Integral = 0
        for i in range(len(self.__First)):
            if time1 >= self.__First[i] and time2 < self.__Second[i] :
                Integral += self.__Constants[i]*(time2 - time1)
            elif time1 >= self.__First[i] and time2 > self.__Second[i]:
                Integral += self.__Constants[i]*(self.__Second[i] - time1)
            elif time1 <= self.__First[i] and time2 > self.__Second[i]:
                Integral += self.__Constants[i]*(self.__Second[i] - self.__First[i])
            elif time1 <= self.__First[i] and time2 < self.__Second[i]:
                Integral += self.__Constants[i]*(time2 - self.__First[i])
                break                
            else:
                pass          
        
        return Integral
    
    def IntegralSquare(self, time1, time2):
        IntegralSquare = 0
        for i in range(len(self.__First)):
            if time1 >= self.__First[i] and time2 < self.__Second[i] :
                IntegralSquare += self.__ConstantsSquare[i]*(time2 - time1)
            elif time1 >= self.__First[i] and time2 > self.__Second[i]:
                IntegralSquare += self.__ConstantsSquare[i]*(self.__Second[i] - time1)
            elif time1 <= self.__First[i] and time2 > self.__Second[i]:
                IntegralSquare += self.__ConstantsSquare[i]*(self.__Second[i] - self.__First[i])
            elif time1 <= self.__First[i] and time2 < self.__Second[i]:
                IntegralSquare += self.__ConstantsSquare[i]*(time2 - self.__First[i])
                break   
            else:
                pass          
        
        return IntegralSquare

Let's test it.

In [146]:
constants = np.array([0,1])
first = np.array([0,0.5])
second = np.array([0.5,1])
my_param = ParametersPiecewiseConstant(constants,first,second)

In [144]:
my_param.Integral(0,0.75)

0.25

In [165]:
my_param.IntegralSquare(0,0.75)

0.25

Now let's test with another inputs.

In [208]:
constants = np.array([0,1,2])
first = np.array([0,0.5,1])
second = np.array([0.5,1,2])
my_param = ParametersPiecewiseConstant(constants,first,second)

In [209]:
my_param.Integral(0,1.5)

1.5

In [210]:
my_param.IntegralSquare(0,1.5)

2.5

In [211]:
my_param.RootMeanSquare(0,1.5)

1.0